In [16]:
from langchain_groq import ChatGroq
import os 


In [17]:
key=os.getenv("groq_api_key")

In [ ]:
llm = ChatGroq(
    temperature=0, 
    GROQ_API_KEY=key,
    model_name="llama-3.3-70b-versatile"
)


In [19]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://careers.nike.com/senior-information-security-analyst/job/R-74144")
page_data=loader.load().pop().page_content
print(page_data[:200])





















Senior Information Security Analyst











































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


In [20]:
from langchain.prompts import PromptTemplate
prompt_text=PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """

)
chain=prompt_text|llm
chain_rest=chain.invoke({'page_data':page_data})
type(chain_rest.content)

str

In [21]:
from langchain_core.output_parsers import JsonOutputParser
parser=JsonOutputParser()
json_res=parser.parse(chain_rest.content)

In [22]:
chain_rest

AIMessage(content='```json\n{\n  "role": "Senior Information Security Analyst",\n  "experience": "Minimum of 3 years relevant IT experience",\n  "skills": [\n    "Certifications such as CISA, CRISC, and CISSP",\n    "Experience in a variety of information technology disciplines",\n    "Experience with local Cyber Security Regulations (PIPL, MLPS, DSL, etc.)",\n    "Former system administration experience, IT audit, or risk advisory experience",\n    "Excellent analytical and problem-solving skills",\n    "Strong attention to detail",\n    "Excellent collaboration skills",\n    "Exceptional communication skills"\n  ],\n  "description": "We’re looking for a Senior Information Security Analyst to leverage knowledge of security policies, standards, controls, and industry best practices to consult with partners across all of Nike Inc. The individual will play a critical role in ensuring that GRC functions are incorporated into key security services and program while validating risk mitigati

In [23]:
json_res

{'role': 'Senior Information Security Analyst',
 'experience': 'Minimum of 3 years relevant IT experience',
 'skills': ['Certifications such as CISA, CRISC, and CISSP',
  'Experience in a variety of information technology disciplines',
  'Experience with local Cyber Security Regulations (PIPL, MLPS, DSL, etc.)',
  'Former system administration experience, IT audit, or risk advisory experience',
  'Excellent analytical and problem-solving skills',
  'Strong attention to detail',
  'Excellent collaboration skills',
  'Exceptional communication skills'],
 'description': 'We’re looking for a Senior Information Security Analyst to leverage knowledge of security policies, standards, controls, and industry best practices to consult with partners across all of Nike Inc. The individual will play a critical role in ensuring that GRC functions are incorporated into key security services and program while validating risk mitigation functions within NIKE Inc. are functioning correctly.'}

In [24]:
import pandas as pd
df=pd.read_csv('my_portfolio.csv')

In [25]:
df.head(4)

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio


In [26]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [30]:
job

{'role': 'Senior Information Security Analyst',
 'experience': 'Minimum of 3 years relevant IT experience',
 'skills': ['Certifications such as CISA, CRISC, and CISSP',
  'Experience in a variety of information technology disciplines',
  'Experience with local Cyber Security Regulations (PIPL, MLPS, DSL, etc.)',
  'Former system administration experience, IT audit, or risk advisory experience',
  'Excellent analytical and problem-solving skills',
  'Strong attention to detail',
  'Excellent collaboration skills',
  'Exceptional communication skills'],
 'description': 'We’re looking for a Senior Information Security Analyst to leverage knowledge of security policies, standards, controls, and industry best practices to consult with partners across all of Nike Inc. The individual will play a critical role in ensuring that GRC functions are incorporated into key security services and program while validating risk mitigation functions within NIKE Inc. are functioning correctly.'}

In [27]:
job=json_res
job['skills']

['Certifications such as CISA, CRISC, and CISSP',
 'Experience in a variety of information technology disciplines',
 'Experience with local Cyber Security Regulations (PIPL, MLPS, DSL, etc.)',
 'Former system administration experience, IT audit, or risk advisory experience',
 'Excellent analytical and problem-solving skills',
 'Strong attention to detail',
 'Excellent collaboration skills',
 'Exceptional communication skills']

In [28]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}]]

In [29]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at SharpCoders. SharpCoders is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of SharpCoders 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase SharpCoders's portfolio: {link_list}
        Remember you are Mohan, BDE at SharpCoders. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Information Security Solutions for Nike Inc.

Dear Hiring Manager,

I came across the job posting for a Senior Information Security Analyst at Nike Inc. and was impressed by the company's commitment to ensuring the security and integrity of its systems. As a Business Development Executive at SharpCoders, I believe our team can provide the necessary expertise to support your information security needs.

At SharpCoders, we have a proven track record of delivering tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has experience in a variety of information technology disciplines, including cybersecurity, IT audit, and risk advisory. We are confident that our skills and expertise align with the requirements outlined in the job posting, including experience with local Cyber Security Regulations (PIPL, MLPS, DSL, etc.) and excellent analytical and problem-solving skills.

Our portfolio showc